In [1]:
import pandas as pd

from MovieEnv import MovieEnv
from PPO import PPO
from Buffer import Experience 

In [ ]:
def main():
    df = pd.read_csv(
    '/home/user/Documents/Python_Workspace/Movie_Project/data//netflix_uk.csv')
    N = 4  # Number of last movies watched kept as state.
    envi = MovieEnv(df, N=N, alpha=1, beta=0)

    lr = 1e-4  # Same learning rate for all modules.
    gamma = 0.95  # Discount factor.
    in_features = 128
    hidden_size = 256
    embedding_dim = 64
    id_list = df['movie_id'].unique()  # List of movies.
    K_epochs = 5
    eps_clip = 0.2
    top_k = 0

    a = PPO(in_features=in_features, out_features=envi.action_space.n, hidden_size=hidden_size, embedding_dim=embedding_dim, lr_actor=lr,
            lr_critic=lr, lr_sr=lr, gamma=gamma, N=N, K_epochs=K_epochs, eps_clip=eps_clip, movie_id_list=id_list, top_k=top_k,
            algo='monte_carlo')

    episodes = 1000
    rewards = []

    for k in range(episodes):
        state, info = envi.reset()
        done = False

        total_reward = 0
        steps_done = 0

        while not done:
            action, state_value, action_logprob = a.select_action(state)
            next_state, reward, done, _, _ = envi.step(action)

            # exp = Experience(state, action, reward, next_state, done, state_value, action_logprob)
            a.add_experience(Experience(state, action, reward,
                            next_state, done, state_value, action_logprob))

            steps_done += 1
            total_reward += reward

            state = next_state

            if (steps_done > 5000):
                break

        print(reward)
        a.learn()
        rewards.append(reward)

if __name__ == "__main__":
    main()

